#  Syft Duet - Alice 

In [1]:
# the length of iris dataset is 150
# how many samples would you let Alice hold?
data_owner = "Alice"
data_size = 70

In [2]:
#check data
if data_size <= 0:
    data_size=1
if data_size > 150:
    data_size=150

## PART 1: Launch a Duet Server and Connect

As a Data Owner, you want to allow someone else to perform data science on data that you own and likely want to protect.

In order to do this, we must load our data into a locally running server within this notebook. We call this server a "Duet".

To begin, you must launch Duet and help your Duet "partner" (a Data Scientist) connect to this server.

You do this by running the code below and sending the code snippet containing your unique Server ID to your partner and following the instructions it gives!

--from openmind course

In [3]:
#launch the server
import syft as sy
duet = sy.launch_duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...


/home/fjunyuan/.local/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/home/fjunyuan/.local/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!

<class 'torch.Tensor'>  -  Objects: 0  Requests: 0   Messages: 0  Request Handlers: 0                                


If you are in Jupyter Notebook (not Colab) the ☝🏾DUET LIVE STATUS above will animate 

# Part 2: Upload data to Duet Server

Let's say the data owner has a dataset of Iris flowers. He will upload the data to the duet server for other data scientists to use.

In [4]:
from sklearn import datasets
import torch as th

In [5]:
iris = datasets.load_iris()
X, y = iris.data, iris.target

In [6]:
print(type(X))

<class 'numpy.ndarray'>


In [7]:
#adjust size
if data_owner == "Alice":
    X = X[0:data_size]#Bob X  = X[(150-data_size):]
    y = y[0:data_size]#Bob y  = y[(150-data_size):]
if data_owner == "Bob":
    X  = X[(150-data_size):]
    y  = y[(150-data_size):]

For doing machine learning using torch, we need the data to be converted to FloatTensors. Here, the data owner is explicitly doing the conversion before uploading the data. If he doesn't do that, it has to be converted in the data scientist's end as you've seen in the previous exercise.

In [8]:
X = th.FloatTensor(X)
y = th.FloatTensor(y)

In [9]:
print(type(X))

In [10]:
#add tag and describtion
X = X.tag("Alice_iris-data")
y = y.tag("Alice_iris-target")

X = X.describe(
    f"This is a dataset for flower classification of {data_size} samples. 4 Features are sepal length (cm)," 
    "sepal width (cm), petal length (cm), petal width (cm)"
)
y = y.describe("Labels for flowers: Iris-setosa, Iris-versicolour, Iris-virginica")

In [11]:
#send pointable dataset to server, data scientist can access to the dataset only if (s)he make a request
data_pointer = X.send(duet, pointable=True)
target_pointer = y.send(duet, pointable=True)

In [12]:
# Once uploaded, the data owner can see the object stored in the tensor
duet.store

[<syft.proxy.torch.TensorPointer object at 0x7f04cc945130>, <syft.proxy.torch.TensorPointer object at 0x7f04cc945340>]

In [13]:
# To see it in a human-readable format, data owner can also pretty-print the tensor information
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 75cd6246ecb84b9eb4b91d9d0cfe1d11>,[Alice_iris-data],This is a dataset for flower classification of...,<class 'torch.Tensor'>
1,<UID: 3470eff444fb47d8adbd497582b6b303>,[Alice_iris-target],"Labels for flowers: Iris-setosa, Iris-versicol...",<class 'torch.Tensor'>


## PART 3: Add Request Handlers

The Data Science Notebook makes a number of requests to access data. You can view these requests with:  
```
duet.requests.pandas
```
Then manually accept or deny them with:
```
duet.requests[0].accept()
```
However for training loops, this can be slow and tedious so the below code will create request handlers which will automatically respond with `accept` or `deny` depending on the name of the request.

In [14]:
duet.requests.pandas

""


In [15]:
# currently we do not have any protection from the handler, handler passes any requests
duet.requests.add_handler(
    action="accept",
    print_local=True  # print the result in your notebook
)

[2022-07-11T01:21:22.310170+0300][CRITICAL][logger]][1411800] > HANDLER Request <UID: 736c006a36c9463f9ee79e8dfce6814c>: To evaluate training progress
Value: <Storable: 1.167180061340332>
[2022-07-11T01:21:23.434340+0300][CRITICAL][logger]][1411800] > HANDLER Request <UID: 5a9078e1fe094031927dbd65b435168c>: To evaluate training progress
Value: <Storable: 1.3690968751907349>
[2022-07-11T01:21:24.551484+0300][CRITICAL][logger]][1411800] > HANDLER Request <UID: 3bfbdd26645444048725130067925ef1>: To evaluate training progress
Value: <Storable: 0.8800439238548279>
[2022-07-11T01:21:25.650693+0300][CRITICAL][logger]][1411800] > HANDLER Request <UID: d87fbd681a2049d283a3f92af8d9eb51>: To evaluate training progress
Value: <Storable: 0.8253083825111389>
[2022-07-11T01:21:25.801131+0300][CRITICAL][logger]][1411800] > HANDLER Request <UID: 3da4469d440e4ba49e205fcedef29d05>: To evaluate training progress
Value: <Storable: 0.68745356798172>
[2022-07-11T01:21:26.913336+0300][CRITICAL][logger]][14118

[2022-07-11T01:21:56.652960+0300][CRITICAL][logger]][1411800] > HANDLER Request <UID: ee8b9250f5d6485ea0562b241a1b5260>:
Value: <Storable: 'in_features=4, out_features=20, bias=True'>
[2022-07-11T01:21:56.785681+0300][CRITICAL][logger]][1411800] > HANDLER Request <UID: 6dc6e84a2b0449baa237dfc1391bdc7f>:
Value: <Storable: OrderedDict([('weight', tensor([[-0.4225, 0.3374, 0.0955, -0.3783],    [-0.2145, -0.4286, 0.1888, 0.0241],    [-0.0034, -0.4554, -0.3730, 0.4715],    [-0.5136, -0.0627, -0.4277, -0.2025],    [-0.0439, -0.2307, -0.4877, -0.0907],    [-0.2953, -0.1715, 0.2471, 0.3373],    [-0.5266, -0.1974, -0.2717, -0.2336],    [-0.2935, -0.4091, -0.3059, -0.2042],    [ 0.3881, -0.4238, 0.1426, 0.2550],    [ 0.0807, 0.6251, -0.4640, -0.2827],    [-0.4040, -0.0901, -0.1418, 0.2642],    [-0.1555, -0.0730, -0.2259, -0.0130],    [ 0.1467, 0.0718, 0.1737, -0.0053],    [-0.3308, 0.3238, -0.2076, 0.1349],    [-0.2804, -0.4949, 0.1882, 0.3707],    [ 0.3147, 0.8142, -0.4489, -0.3328],    [-0.057

[2022-07-11T01:22:02.317608+0300][CRITICAL][logger]][1411800] > HANDLER Request <UID: c515dd693a4f4792a991873f53f0fd86>:
Value: <Storable: 'in_features=30, out_features=3, bias=True'>
[2022-07-11T01:22:03.428483+0300][CRITICAL][logger]][1411800] > HANDLER Request <UID: c792ef2e24db4e62b9dfb6b4cfa02fa2>:
Value: <Storable: OrderedDict([('weight', tensor([[ 0.3440, 0.2581, -0.3220, 0.4146, 0.4303, 0.0319, -0.2318, -0.0182,     -0.4661, -0.0672, -0.1751, -0.1455, 0.3218, -0.1694, -0.0947, 0.1031,     0.0838, 0.2976, -0.3621, 0.3640, -0.3272, 0.0889, -0.2386, -0.0975,     -0.2137, 0.3018, 0.4223, -0.0330, -0.1699, -0.0179],    [-0.0965, -0.5465, 0.2861, -0.4124, -0.2538, 0.1219, 0.2790, 0.3581,     0.2567, 0.3852, 0.1546, 0.0877, -0.3155, 0.3479, -0.1120, -0.0192,     0.0251, 0.0250, 0.3501, -0.3604, 0.1715, -0.1475, 0.1407, 0.3140,     0.4434, -0.3726, -0.5917, 0.3892, 0.4410, -0.1959],    [-0.5126, -0.5064, -0.3504, -0.2566, -0.3079, -0.0508, -0.3666, -0.9498,     -0.4580, -0.3527, 0.0621

[2022-07-11T01:22:34.699166+0300][CRITICAL][logger]][1411800] > HANDLER Request <UID: fd189e9b7e284aa0afc1ac0c832d2f8f>: To run test and inference locally
Value: <Storable: 'in_features=30, out_features=30, bias=True'>
[2022-07-11T01:22:35.822485+0300][CRITICAL][logger]][1411800] > HANDLER Request <UID: a4b04f5b1eb8434cac6b6e71ae8bd663>: To run test and inference locally
Value: <Storable: OrderedDict([('weight', tensor([[-0.1245, 0.1009, -0.2209, -0.1519, 0.1356, 0.0399, -0.0098, 0.0216,     0.1111, 0.0934, 0.0311, 0.5022, 0.0590, 0.1102, 0.0097, 0.1349,     0.1188, -0.0497, -0.4548, 0.2428, -0.1132, -0.1102, -0.1232, 0.0987,     -0.0780, 0.0429, 0.1604, 0.3102, 0.4151, 0.1458],    [ 0.1350, 0.1027, -0.5394, -0.4240, 0.1089, 0.1567, -0.1587, -0.1087,     0.1437, -0.4384, -0.1728, -0.5542, -0.5178, -0.2473, -0.1549, -0.3811,     -0.5227, -0.5218, -0.5508, -0.4526, 0.1290, -0.3200, -0.5482, -0.4055,     -0.5113, -0.4090, 0.1642, 0.0722, -0.2950, -0.4899],    [ 0.0887, 0.0177, -0.5674, -0

[2022-07-11T01:22:36.920067+0300][CRITICAL][logger]][1411800] > HANDLER Request <UID: 85a7e6216f7746adbde5842b13a8978a>: To run test and inference locally
Value: <Storable: 'in_features=30, out_features=3, bias=True'>
[2022-07-11T01:22:38.016867+0300][CRITICAL][logger]][1411800] > HANDLER Request <UID: c9084518ae5f4ddb95945ffd741eaa4e>: To run test and inference locally
Value: <Storable: OrderedDict([('weight', tensor([[ 0.3440, 0.2581, -0.3220, 0.4146, 0.4303, 0.0319, -0.2318, -0.0182,     -0.4661, -0.0672, -0.1751, -0.1455, 0.3218, -0.1694, -0.0947, 0.1031,     0.0838, 0.2976, -0.3621, 0.3640, -0.3272, 0.0889, -0.2386, -0.0975,     -0.2137, 0.3018, 0.4223, -0.0330, -0.1699, -0.0179],    [-0.0965, -0.5465, 0.2861, -0.4124, -0.2538, 0.1219, 0.2790, 0.3581,     0.2567, 0.3852, 0.1546, 0.0877, -0.3155, 0.3479, -0.1120, -0.0192,     0.0251, 0.0250, 0.3501, -0.3604, 0.1715, -0.1475, 0.1407, 0.3140,     0.4434, -0.3726, -0.5917, 0.3892, 0.4410, -0.1959],    [-0.5126, -0.5064, -0.3504, -0.25